#**Big Data Application in E-commense**
##——Customer Behavior analysis and recommendation
###Project Contributor : **Tao Liu**



In this project, we will use [Amazon Review Data](http://deepyeti.ucsd.edu/jianmo/amazon/index.html) and perform different kind of data anlysis methods to analyze the customer behaviors for buying good and then give recommendation based on analysis.

##**Step 0** - Package import
All the packages will be imported here.

In [52]:
import numpy as np
import sklearn
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import requests
import array
import collections

##**Step 1** - Data Implemtation and cleaning

We will start with Amazon Review Data implementation and cleaning them if possible.



In [3]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Gift_Cards.json.gz

--2020-11-16 17:46:05--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Gift_Cards.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 380174 (371K) [application/octet-stream]
Saving to: ‘meta_Gift_Cards.json.gz.1’

meta_Gift_Cards.jso 100%[===================>] 371.26K  1.76MB/s    in 0.2s    

2020-11-16 17:46:06 (1.76 MB/s) - ‘meta_Gift_Cards.json.gz.1’ saved [380174/380174]



In [36]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
      yield json.dumps(eval(l))
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
      df[i] = d
      i += 1
    return pd.DataFrame.from_dict(df, orient='index')
def build_database(asin_dict):
    f = open("output.strict", 'w')
    for l in parse("meta_Gift_Cards.json.gz"):
          (f.write(l + '\n'))
    df = getDF('meta_Gift_Cards.json.gz')[0]
    length = len(df)
    n = 2000
    #also_view
    also_view_dict ={}
    also_view = []
    for i in df:
          dictionary = json.loads(i)
          asin =dictionary['asin']
          asin = asin_dict[asin]
          also_view_raw = dictionary['also_view']
          sub_list1=[]
          for j in also_view_raw:
                try:
                    sub_list1.append(asin_dict[j])
                    also_view_dict.setdefault(asin_dict[j],[]).append(asin)
                except:
                    sub_list1.append(n)
                    also_view_dict.setdefault(n,[]).append(asin)
                    asin_dict[j] = n
                    n = n+1
          also_view.append(sub_list1)
    #also_buy
    also_buy_dict ={}
    also_buy = []
    for i in df:
          dictionary = json.loads(i)
          asin =dictionary['asin']
          asin = asin_dict[asin]
          also_buy_raw = dictionary['also_buy']
          sub_list2=[]
          for k in also_buy_raw:
                try:
                    sub_list2.append(asin_dict[k])
                    also_buy_dict.setdefault(asin_dict[k],[]).append(asin)
                except:
                    sub_list2.append(n)
                    also_buy_dict.setdefault(n,[]).append(asin)
                    asin_dict[k] = n
                    n = n+1
          also_buy.append(sub_list2)  
    return  also_view,also_buy,also_view_dict,also_buy_dict,asin_dict
# no longer used
def to_CSV(raw_database):
    dataframe=pd.DataFrame(data=raw_database)
    dataframe.to_csv('database.csv',encoding='gbk')

def reformat_asin():
    f = open("output.strict", 'w')
    for l in parse("meta_Gift_Cards.json.gz"):
          (f.write(l + '\n'))
    df = getDF('meta_Gift_Cards.json.gz')[0]
    asin_dict = {}
    n = 0
    for i in df:
          dictionary = json.loads(i)
          asin = dictionary['asin']
          asin_dict[asin] = n
          n = n+1
    return asin_dict


asin_dict = reformat_asin()
print(len(asin_dict))
also_view,also_buy,also_view_dict,also_buy_dict,asin_dict = build_database(asin_dict)
print(also_view)
print(also_buy)
print(len(also_view_dict)) 
print(also_buy_dict)
print(len(also_buy)-len(also_view))
print("This is the length of the asin_dict",len(asin_dict))

1547
[[], [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025], [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 922, 2036, 283, 2037, 2038, 2001, 2000, 2013, 2039, 2040, 2007, 2041, 2042, 2009, 1233, 2043, 2014, 234, 2044, 2012, 2005, 2045, 2046, 2002, 2047, 378, 2048, 1382, 2008, 2004, 2011, 2049, 2050, 2051, 2052, 2003, 2053, 2054, 2018, 2055, 2056], [], [], [], [], [], [], [], [2057, 2058, 2059], [2060, 2061, 2000, 2041], [2000], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [2014, 2000, 2001, 2004], [], [], [], [2014], [], [], [], [], [2014, 2000, 2004, 2062, 234, 2063, 2013, 1490, 2009, 2002, 2006, 2001, 2064], [], [], [2014, 2000, 2017, 2002], [], [2014, 2000], [2000, 2014, 2002, 2004], [2014, 2000], [], [], [], [], [], [], [], [2014], [], [], [], [], [], [], [

##**Step 2** - Recommendation Rate Calculation

https://blog.csdn.net/huagong_adu/article/details/7362908

if customer purchase item 1, and item 1 appears in the also_like or also_view list in item 2, then the item 2's similarity to item 1 will be affected, otherwise the mean value will be considered.


In [68]:
from sklearn.metrics.pairwise import cosine_similarity
def itemtoitem_relationship(also_view,also_buy,also_view_dict,also_buy_dict,asin_dict):
  matrix =[]
  for itemNO in range(0,1547):
      asin_list = [0]*len(asin_dict)
      for asin in also_view[itemNO]:
          try:
            asin_list[asin] = asin_list[asin]+2
          except:
            print(len(asin_list))
          try:
              for i in also_view_dict[asin]:
                asin_list[i] = asin_list[i]+2
          except:
              pass
      for asin in also_buy[itemNO]:
          try:
               for i in also_buy_dict[asin]:
                asin_list[i] = asin_list[i]+10
          except:
              pass
      try:
        avg = (len(also_view[itemNO]) + len(also_buy[itemNO]))/2
      except:
        avg = 0
      asin_list = [i if i != 0 else avg for i in asin_list]
      matrix. append(asin_list)
  return matrix
matrix = itemtoitem_relationship(also_view,also_buy,also_view_dict,also_buy_dict,asin_dict)

##**Step 3** - Recommendation Strategy similarity Calculation


In [69]:
cosine = cosine_similarity(matrix)
np.fill_diagonal(cosine, 0 )
similarity_example =pd.DataFrame(cosine)
similarity_example.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537,1538,1539,1540,1541,1542,1543,1544,1545,1546
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,0.0,0.000000,0.928569,0.0,0.0,0.0,0.0,0.0,0.873242,0.0,0.932718,0.895627,0.718031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.957735,0.766296,0.806564,0.940168,0.766296,0.820201,0.879648,0.863465,0.0,0.0,0.901829,0.910500,0.788548,0.814934,0.787332,0.887501,0.821184,0.856774,0.834135,0.791907,0.783412,0.904253,0.910979,0.678149,0.904549,0.898554,0.538294,0.529602,0.549931,0.564857,0.552024,0.728267,0.563946,0.431074,0.0,0.551786,0.366387,0.567856,0.876188
2,0.0,0.928569,0.000000,0.0,0.0,0.0,0.0,0.0,0.916060,0.0,0.930553,0.892680,0.736485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.950722,0.847784,0.892851,0.851972,0.847784,0.904136,0.931512,0.921625,0.0,0.0,0.944907,0.809825,0.900110,0.911231,0.858773,0.953365,0.907724,0.933473,0.921867,0.902232,0.898392,0.948418,0.957183,0.747432,0.949056,0.954262,0.653532,0.646918,0.664976,0.687954,0.672772,0.834744,0.683250,0.516743,0.0,0.673049,0.454307,0.690120,0.933010
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


##**Step 4** - Recommendation Strategy Showcase

In [87]:
trainingdata  = itemtoitem_relationship(also_view,also_buy,also_view_dict,also_buy_dict,asin_dict)
database = ["B00H5BPJQC","B00GOLH1LK","B002NZXF9S"]
def recommedndation_based_on_purchased(database):
  possibility_list =[1]*1547
  for i in database:
    internal_code = asin_dict[i]
    possibility_list = [a + b for a, b in zip(possibility_list, cosine[internal_code])]
  max = -1
  for i in range(0,len(possibility_list)):
    if possibility_list[i] > max:
      position = i
      max = possibility_list[i]
    else:
      pass
  return list(asin_dict.keys())[list(asin_dict.values()).index(position)]
recommedndation_based_on_purchased(database)

'B00KF5MK08'